In [1]:
import sys
import os

# 프로젝트 루트 디렉토리 경로를 추가
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [2]:
import jax
import jax.numpy as jnp
import chex
import numpy as np
#disable jax JIT
#jax.config.update("jax_disable_jit", True)

from tqdm.autonotebook import tqdm
from JAxtar.hash import hash_func_builder, HashTable
from JAxtar.bgpq import BGPQ, HashTableIdx_HeapValue
from JAxtar.astar import astar_builder
from functools import partial
from puzzle.slidepuzzle import SlidePuzzle
from heuristic.slidepuzzle_heuristic import SlidePuzzleHeuristic

/tmp/ipykernel_2540/2469263457.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
puzzle = SlidePuzzle(4)

In [4]:
states = jax.vmap(puzzle.get_initial_state, in_axes=0)(key=jax.random.split(jax.random.PRNGKey(32),1))
#states = puzzle.State(board=jnp.array([0, 12, 9, 13, 15, 11, 10, 14, 3, 7, 2, 5, 4, 8, 6, 1], dtype=jnp.uint8))[jnp.newaxis, ...]
target = puzzle.State(board=jnp.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0], dtype=jnp.uint8))

In [5]:
size = int(1e6)
batch_size = int(1000)
astar_fn = astar_builder(puzzle, SlidePuzzleHeuristic(puzzle).distance, batch_size, size)

states, filled = HashTable.make_batched(puzzle.State, states, batch_size)
astar_result, hash_idxs, last_states, solved = astar_fn(states, filled, target)

In [6]:
print(last_states[jnp.argmax(solved)][0])

┏━━━┳━━━┳━━━┳━━━┓
┃ 1 ┃ 2 ┃ 3 ┃ 4 ┃
┣━━━╋━━━╋━━━╋━━━┫
┃ 5 ┃ 6 ┃ 7 ┃ 8 ┃
┣━━━╋━━━╋━━━╋━━━┫
┃ 9 ┃ A ┃ B ┃ C ┃
┣━━━╋━━━╋━━━╋━━━┫
┃ D ┃ E ┃ F ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛


In [7]:
astar_result.hashtable.size

Array(350146, dtype=int32)

In [8]:
print(astar_result.hashtable.table[hash_idxs[0].index, hash_idxs[0].table_index][0])

┏━━━┳━━━┳━━━┳━━━┓
┃ C ┃ 1 ┃ B ┃ E ┃
┣━━━╋━━━╋━━━╋━━━┫
┃ A ┃ 5 ┃ 6 ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃ 8 ┃ F ┃ 9 ┃ D ┃
┣━━━╋━━━╋━━━╋━━━┫
┃ 3 ┃ 4 ┃ 2 ┃ 7 ┃
┗━━━┻━━━┻━━━┻━━━┛


In [9]:
astar_result, hash_idxs, states, solved = astar_fn(states, filled, target)